In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master('yarn')\
        .appName('trip_count_sql')\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-06-15 01:27:00,683 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-06-15 01:27:04,470 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
# Master가 yarn으로 잘 되어있는지 확인
spark

In [3]:
# hdfs 상에 존재하는 파일의 경로
filepath = '/user/ubuntu/taxi/taxi_202003.csv'
taxi_df = spark.read.csv(f'hdfs://{filepath}',
                        inferSchema = True,
                        header = True)

taxi_df.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   null|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   null|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   null|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   null|
+-----------------+--------------------+-------------------+-------------------+

In [4]:
taxi_df.createOrReplaceTempView('mobiliy_data')

In [5]:
# 승차 년-월-일 별 카운트 세기
query = """
SELECT pickup_date, count(*) as trips

FROM (
        SELECT split(pickup_datetime, ' ')[0] as pickup_date
        FROM mobility_data
     )

GROUP BY pickup_date
"""
spark.sql(query).show()

AnalysisException: Table or view not found: mobility_data; line 6 pos 13;
'Aggregate ['pickup_date], ['pickup_date, count(1) AS trips#67L]
+- 'SubqueryAlias __auto_generated_subquery_name
   +- 'Project ['split('pickup_datetime,  )[0] AS pickup_date#66]
      +- 'UnresolvedRelation [mobility_data], [], false
